<div class="text-muted">

# Lecture Notes

Instructions:

> **Week k (week that group topics are presented in lecture)** <br>
By Friday at 4pm, the group is required to submit their typed-up notes from that week’s
lecture. This will form the basic outline of their deliverable and they will receive
feedback from the TA on their work. <br><br>
The group should also have an outline of the one outside paper that they intend to
describe in greater detail.

## L13. Constraint Programming
- Big picture
- Viewing Sodoku and n-Queens as CSPs
- Constraint Modeling
- Applications of CSPs
- Tying to other course content

## L14. Solving Constraint Programs - Propagation and Basic Search
- Constraint propagation: Achieving arc consistency with AC-1 and AC-3
- Examples on integer domains with algebra
- Complexity of AC-1 and AC-3
- Arc consistency: sound but not complete

## L15. Solving Constraint Programs - Backtrack Search and Forward Checking
- Narrative: show the largest reasonable n-Queens problem that can be solved for various solution methods
- Combining inference with search
    - Why standard search sucks
    - Backtrack search
- Constraint Optimization Problem: Branch & Bound
- Forward Checking: Combining backtracking with **limited** constraint propagation
- Dynamic variable ordering and BT-FC
- "Bucket Elimination", a form of adaptive consistency (i.e., variable elimination)
    - Viewing arc consistency as variable elimination

###### Optional Content, Not Discussed in Lecture
- Adaptive consistency is linear for trees
- Iterative repair
- Conflict-directed back jumping

</div>

---

# Constraint Programming
Topics: constraint propagation, search, variable elimination; global constraints; MVRP

## 1. Introduction

Constraints naturally arise in a variety of interactions and fields of study such as game theory, social studies, operations research, engineering, and artificial intelligence. A constraint refers to the relationship between the state of objects, such as the constraint that the three angles of a triangle must sum to 180 degrees. Note that this constraint has not precisely stated each angle's value and still allows some flexibility. Said another way, the triangle constraint restricts the values that the three variables (each angle) can take, thus providing information that will be useful in finding values for the three angles.

Another example of a constrained problem comes from the recently-aired hit TV series *Buddies*, where a group of five (mostly mutual) friends would like to sit at a table with three chairs in specific arrangements at different times, but have requirements as to who they will and will not sit with.

Another example comes from scheduling: at the university level, there is a large number of classes that must be scheduled in various classrooms such that no professor or classroom is double booked. Further, there are some constraints on which classes can be scheduled for the same time, as some students will need to be registered for both.

Computers can be employed to solve these types of problems, but in general these tasks are computationally intractible and cannot be solved efficiently in all cases with a single algorithm [1]. However, by formalizing these types of problems in a constraint processing framework, we can identify classes of problems that can be solved using efficient algorithms.

### 1.1. Modelling

A **constraint satisfaction problem** (CSP) is formalized by a *constraint network*, which is the triple $\langle V,D,C\rangle$, where
- $V = \{V_i\}_{i=1}^n$ is the set of $n$ variables
- $D = \{D_i\}_{i=1}^n$ is the set of variable domains, where the domain of variable $V_k$ is $D_k$
- $C = \{C_i\}_{i=1}^m$ is the set of constraints on the values that each $V_i$ can take on. Specifically,
    - Each constraint $C_i = \langle S_i,R_i\rangle$ specifies allowed variable assignments.
    - $S_i \subset V$ contains the variables involved in the constraint, called the *scope* of the constraint.
    - $R_i$ is the constraint's *relation* and represents the simultaneous legal value assignments of variables in the associated scope.
        - For example, if scope of the first constraint is $S_1 = \{V_3, V_8\}$, then the relation $R_1$ is a subset of the Cartesian product of those variables' domains: $R_1 \subset D_3 \times D_8$, and an element of the relation $R_1$ could be written as a 2-tuple $(x,y)\in R_1$.

Note that for a CSP model, *any* consistent assignment of the variables (i.e., where all constraints are satisfied) constitutes a valid solution, it may not be the "best" solution. Notions of optimality can be captured by introducing an objective function which is used to find a valid solution with the lowest cost. This is referred to as a **constraint *optimization* problem** (COP). We will refer generally to CSPs with the understanding that a CSP can easily become a COP.

### 1.1.1. Modelling as a Graph
<div class="alert alert-danger">
TODO: Discuss the graphical view of constraints. Let's also assume that we are only considering binary CSPs and note that any n-connected CSP can be reformulated into an equivalent binary CSP.
</div>

### 1.2. Solving

The goal of formalizing a CSP as a constraint network model is to efficiently solve it using computational algorithms and tools. **Constraint programming** (CP) is a powerful tool to solve combinatorial constraint problems and is the study of computational systems based on constraints. Once the problem has been modeled as a formal CSP, a variety of computable algorithms could be used to find a solution that satisfies all constraints. We will restrict ourselves to using CP to solve CSPs with **discrete, finite domains**.

In general, there are two methods used to solve a CSP: search or inference. In previous 16.410/413 problems, **state-space search** was used to find the best path through some sort of graph or tree structure. Likewise, state-space search could be used to find a valid "path" through the CSP that satisfies each of the local constraints and is therefore a valid global solution. However, this approach would quickly become intractable as the number of variables and the size of each of their domains increase.

In light of this, the second solution method becomes more attractive. **Constraint propagation**, a specific type of inference, is used to reduce the number of legal values from a variable's domain by pruning values that would violate the constraints of the given variable. By making a variable locally consistent with its constraints, the domain of adjacent variables may potentially be further reduced as a result of missing values in the pairwise constraint of the two variables. In this way, by making the first variable consistent with its constraints, the constraints of neighboring variables can be re-evaluated, causing a further reduction of domains through the propagation of constraints. These ideas will later be formalized as $k$-consistency.

Constraint propagation may be combined with search, using the pros of both methods simultaneously. Alternatively, constraint propagation may be performed as a pre-processing pruning step so that search has a smaller state space to search over. Sometimes, constraint propagation is all that is required and a solution can be found without a search step at all.

After giving examples of modelling CSPs, this notebook will explore a variety of solution methods based on constraint propagation and search.

---

## 2. Modeling

Given a constrained problem, it is desirable to identify an appropriate constraint network model $\langle V,D,C\rangle$ that can be used to find its solution. Modelling for CSPs is an important step that can dramatically affect the difficulty in enumerating the associated constraints or efficiency of finding a solution.

### 2.1. Examples

In this section, we consider two puzzle problems and model them as CSPs.

<div class="alert alert-danger">
TODO: Make these models more formal?
</div>

### 2.1.1. Sudoku

Sudoku is a well-known puzzle that can be easily solved using the CSP framework. Given a $9\times 9$ grid partially filled with numbers from $1$ to $9$, the goal is to fill the grid entirely such that:

- each $3\times 3$ sub-grid contains all numbers from $1$ to $9$
- each row contains all numbers from $1$ to $9$
- each column contains all numbers from $1$ to $9$

<table><tr>
<td> <img src="images/start_sudoku.png"/> </td>
<td> <img src="images/end_sudoku.png" /> </td>
</tr></table>

### 2.1.2. N-Queens

The N-queens problem is a well-know puzzle among computer scientists. Given any integer $N$, the goal is to place $N$ queens on an $N\times N$ chessboard satisfying the constraint that no two queens threaten each other. A queen can threaten any other queen that is on the same row, column, or diagonal.

<img src="images/nqueens.png" align="center"/>

---

## 3. Constraint Propagation


---

## 4. Backtrack Search

---

## 5. Hybrid Methods

---

## Bibliography

[1] Rina Dechter. 2003. Constraint Processing. Morgan Kaufmann Publishers Inc., San Francisco, CA, USA.